In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as time
from collections import Counter
from mlxtend.frequent_patterns import apriori, association_rules
#pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini_2.parquet',engine='pyarrow')
#df = df[(~df.reparto.isin(["95","80","85"]))&(df.tipo=="")]
df

In [ ]:
df.reparto.unique()

In [ ]:
def hot_encode(x):
    if(x==0):
        return False
    else:
        return True

In [ ]:
transazioni=df[['cassa','transazione','data_reg','descrizione','reparto','qta_riga']]

Frequent set dei reparti periodo settebre-gennaio

In [ ]:
frq_items_reparti=[]
for reparto in df.reparto.unique():
    
    basket_rep = transazioni[transazioni.reparto==reparto]\
    .groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga']\
    .sum()\
    .unstack(fill_value=0)\
    .reset_index()\
    .drop(['cassa', 'transazione','data_reg'], axis=1)\
    .applymap(hot_encode)

    frq_items=apriori(basket_rep, min_support = 0.01, use_colnames = True).sort_values(by=["support"],ascending=False)
    frq_items_reparti.append(
    {
        "reparto" : reparto,
        "itemset" : list(frq_items.iloc[0].itemsets)[0],
        "grandezza" : len(basket_rep) * frq_items.iloc[0].support
    }
    )

In [ ]:
pd.DataFrame(frq_items_reparti).plot.bar(figsize=(25,7),x="itemset", y="grandezza")